# Fine tune BART for the 4 different subreddit groupings: 

1. advice_story
1. gaming
1. media_lifestyle_sports
1. other

# Setups

In [1]:
from IPython.display import clear_output

!pip install datasets transformers rouge_score rouge-score nltk
# rouge-score is the google version
!pip install pyarrow
!pip install -q sentencepiece

clear_output()

In [2]:
import os
import re
import time
from tqdm.notebook import trange, tqdm
import pandas as pd
import numpy as np
from pprint import pprint
import matplotlib.pyplot as plt

# nlp stuff
import nltk
nltk.download('punkt')

# tf stuff
import tensorflow_datasets as tfds 
import tensorflow as tf
from transformers import PegasusTokenizer, TFPegasusForConditionalGeneration # pegasus
from transformers import BartTokenizer, TFBartForConditionalGeneration # bart

# pytorch dataset types
import datasets
from datasets.dataset_dict import DatasetDict
from datasets import Dataset, load_metric, load_dataset

# pytorch bart stuff
import torch
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer
from transformers import AutoTokenizer

clear_output()

In [ ]:
# sign into huggingface: https://huggingface.co/settings/tokens
from huggingface_hub import notebook_login
notebook_login()

Login successful
Your token has been saved to /root/.huggingface/token
Authenticated through git-credential store but this isn't the helper defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub. Run the following command in your terminal in case you want to set this credential helper as the default

git config --global credential.helper store


In [ ]:
#!apt install git-lfs

# Load data

In [3]:
# specify your path to the repo here:
repo_path = '/content/gdrive/MyDrive/w266/w266_reddit_summarization'

In [4]:
%%time
from google.colab import drive
drive.mount('/content/gdrive')
data_path = os.path.join(repo_path, 'data/reddit_parquet/train_test_split_v2')
os.chdir(data_path)
files = [i for i in os.listdir(data_path) if re.search("reddit", i)]

train = pd.read_parquet('reddit_train.parquet')
test = pd.read_parquet('reddit_test.parquet')
valid = pd.read_parquet('reddit_validation.parquet')

Mounted at /content/gdrive
CPU times: user 1.24 s, sys: 375 ms, total: 1.61 s
Wall time: 20.1 s


In [ ]:
print("train")
print(train['subreddit_group'].value_counts())

print("\n\ntest:")
print(test['subreddit_group'].value_counts())

print("\n\nvalid:")
valid['subreddit_group'].value_counts()

train
advice/story              15000
gaming                    15000
media/lifestyle/sports    15000
other                     15000
Name: subreddit_group, dtype: int64


test:
advice/story              1000
gaming                    1000
media/lifestyle/sports    1000
other                     1000
Name: subreddit_group, dtype: int64


valid:


advice/story              1000
gaming                    1000
media/lifestyle/sports    1000
other                     1000
Name: subreddit_group, dtype: int64

# Modeling

In [5]:
# bunch of diff checkpoints to consider

# bart checkpoints
# model_checkpoint = 'facebook/bart-base' # keep returning the first sentence for me, extractive.
# model_checkpoint = 'facebook/bart-large-mnli' # same as above, only returns first sentences. extractive.
# model_checkpoint = 'sshleifer/distilbart-cnn-12-6' # works a bit better, but seems to produce extractive summaries still. 
model_checkpoint = 'sshleifer/distilbart-xsum-6-6' # trained on both xsum and cnn/dm

# pegasus checkpoints:
# model_checkpoint = "google/pegasus-xsum" # works  well
# model_checkpoint = 'google/pegasus-reddit_tifu' # also works  well

In [ ]:
# load model, tokenizer, and rouge metric
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)
metric = load_metric("rouge")

clear_output()

Downloading:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.52k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/439M [00:00<?, ?B/s]

In [ ]:
# convert data to torch Dataset. Also split up into 4 groups
raw_datasets = DatasetDict({
    # advice_story
    'train_advice_story': Dataset.from_dict({
        'content': train[train['subreddit_group'] == 'advice/story']['content'],
        'summary': train[train['subreddit_group'] == 'advice/story']['summary'],
        'subreddit': train[train['subreddit_group'] == 'advice/story']['subreddit'],
        'subreddit_group': train[train['subreddit_group'] == 'advice/story']['subreddit_group']
    }), 
    'test_advice_story': Dataset.from_dict({
        'content': test[test['subreddit_group'] == 'advice/story']['content'],
        'summary': test[test['subreddit_group'] == 'advice/story']['summary'],
        'subreddit': test[test['subreddit_group'] == 'advice/story']['subreddit'],
        'subreddit_group': test[test['subreddit_group'] == 'advice/story']['subreddit_group']
    }), 
    'valid_advice_story': Dataset.from_dict({
        'content': valid[valid['subreddit_group'] == 'advice/story']['content'],
        'summary': valid[valid['subreddit_group'] == 'advice/story']['summary'],
        'subreddit': valid[valid['subreddit_group'] == 'advice/story']['subreddit'],
        'subreddit_group': valid[valid['subreddit_group'] == 'advice/story']['subreddit_group']
    }),

    # media_lifestyle_sports
    'train_media_lifestyle_sports': Dataset.from_dict({
        'content': train[train['subreddit_group'] == 'media/lifestyle/sports']['content'],
        'summary': train[train['subreddit_group'] == 'media/lifestyle/sports']['summary'],
        'subreddit': train[train['subreddit_group'] == 'media/lifestyle/sports']['subreddit'],
        'subreddit_group': train[train['subreddit_group'] == 'media/lifestyle/sports']['subreddit_group']
    }), 
    'test_media_lifestyle_sports': Dataset.from_dict({
        'content': test[test['subreddit_group'] == 'media/lifestyle/sports']['content'],
        'summary': test[test['subreddit_group'] == 'media/lifestyle/sports']['summary'],
        'subreddit': test[test['subreddit_group'] == 'media/lifestyle/sports']['subreddit'],
        'subreddit_group': test[test['subreddit_group'] == 'media/lifestyle/sports']['subreddit_group']
    }), 
    'valid_media_lifestyle_sports': Dataset.from_dict({
        'content': valid[valid['subreddit_group'] == 'media/lifestyle/sports']['content'],
        'summary': valid[valid['subreddit_group'] == 'media/lifestyle/sports']['summary'],
        'subreddit': valid[valid['subreddit_group'] == 'media/lifestyle/sports']['subreddit'],
        'subreddit_group': valid[valid['subreddit_group'] == 'media/lifestyle/sports']['subreddit_group']
    }),
    
    # gaming
    'train_gaming': Dataset.from_dict({
        'content': train[train['subreddit_group'] == 'gaming']['content'],
        'summary': train[train['subreddit_group'] == 'gaming']['summary'],
        'subreddit': train[train['subreddit_group'] == 'gaming']['subreddit'],
        'subreddit_group': train[train['subreddit_group'] == 'gaming']['subreddit_group']
    }), 
    'test_gaming': Dataset.from_dict({
        'content': test[test['subreddit_group'] == 'gaming']['content'],
        'summary': test[test['subreddit_group'] == 'gaming']['summary'],
        'subreddit': test[test['subreddit_group'] == 'gaming']['subreddit'],
        'subreddit_group': test[test['subreddit_group'] == 'gaming']['subreddit_group']
    }), 
    'valid_gaming': Dataset.from_dict({
        'content': valid[valid['subreddit_group'] == 'gaming']['content'],
        'summary': valid[valid['subreddit_group'] == 'gaming']['summary'],
        'subreddit': valid[valid['subreddit_group'] == 'gaming']['subreddit'],
        'subreddit_group': valid[valid['subreddit_group'] == 'gaming']['subreddit_group']
    }),

    # other
    'train_other': Dataset.from_dict({
        'content': train[train['subreddit_group'] == 'other']['content'],
        'summary': train[train['subreddit_group'] == 'other']['summary'],
        'subreddit': train[train['subreddit_group'] == 'other']['subreddit'],
        'subreddit_group': train[train['subreddit_group'] == 'other']['subreddit_group']
    }), 
    'test_other': Dataset.from_dict({
        'content': test[test['subreddit_group'] == 'other']['content'],
        'summary': test[test['subreddit_group'] == 'other']['summary'],
        'subreddit': test[test['subreddit_group'] == 'other']['subreddit'],
        'subreddit_group': test[test['subreddit_group'] == 'other']['subreddit_group']
    }), 
    'valid_other': Dataset.from_dict({
        'content': valid[valid['subreddit_group'] == 'other']['content'],
        'summary': valid[valid['subreddit_group'] == 'other']['summary'],
        'subreddit': valid[valid['subreddit_group'] == 'other']['subreddit'],
        'subreddit_group': valid[valid['subreddit_group'] == 'other']['subreddit_group']
    })
})

raw_datasets

DatasetDict({
    train_advice_story: Dataset({
        features: ['content', 'summary', 'subreddit', 'subreddit_group'],
        num_rows: 15000
    })
    test_advice_story: Dataset({
        features: ['content', 'summary', 'subreddit', 'subreddit_group'],
        num_rows: 1000
    })
    valid_advice_story: Dataset({
        features: ['content', 'summary', 'subreddit', 'subreddit_group'],
        num_rows: 1000
    })
    train_media_lifestyle_sports: Dataset({
        features: ['content', 'summary', 'subreddit', 'subreddit_group'],
        num_rows: 15000
    })
    test_media_lifestyle_sports: Dataset({
        features: ['content', 'summary', 'subreddit', 'subreddit_group'],
        num_rows: 1000
    })
    valid_media_lifestyle_sports: Dataset({
        features: ['content', 'summary', 'subreddit', 'subreddit_group'],
        num_rows: 1000
    })
    train_gaming: Dataset({
        features: ['content', 'summary', 'subreddit', 'subreddit_group'],
        num_rows: 15000
  

In [ ]:
# tokenize everything
max_input_length = 1024
max_target_length = 128

def preprocess_function(examples):
    inputs = [doc for doc in examples["content"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples["summary"], max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_datasets = raw_datasets.map(preprocess_function, batched=True)

Parameter 'function'=<function preprocess_function at 0x7f68a35fb560> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


  0%|          | 0/15 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/15 [00:00<?, ?ba/s]

...

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/15 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [ ]:
tokenized_datasets

DatasetDict({
    train_advice_story: Dataset({
        features: ['content', 'summary', 'subreddit', 'subreddit_group', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 15000
    })
    test_advice_story: Dataset({
        features: ['content', 'summary', 'subreddit', 'subreddit_group', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 1000
    })
    valid_advice_story: Dataset({
        features: ['content', 'summary', 'subreddit', 'subreddit_group', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 1000
    })
    train_media_lifestyle_sports: Dataset({
        features: ['content', 'summary', 'subreddit', 'subreddit_group', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 15000
    })
    test_media_lifestyle_sports: Dataset({
        features: ['content', 'summary', 'subreddit', 'subreddit_group', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 1000
    })
    valid_media_lifestyle_sports: Dataset({
        features: ['content

In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    # Rouge expects a newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]
    
    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    # Extract a few results
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}
    
    # Add mean generated length
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)
    
    return {k: round(v, 4) for k, v in result.items()}

# Train - advice_story

- 1st time: epochs=3, obs=20k, batch_size=4. Took 2:02 hrs
- 2nd time: epochs=3, obs=15k, batch_size=8. Took 1:23 hrs

In [ ]:
%%time
# note, batch size of 8 seems to almost max out the gpu
# so probs dont go any higher than this. 
args = Seq2SeqTrainingArguments(
    f"BART_reddit_advice_story", 
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8, # 16
    per_device_eval_batch_size=8, #16
    weight_decay=0.01,
    save_total_limit=1, #3,
    num_train_epochs=3,
    predict_with_generate=True,
    fp16=True,
    push_to_hub=True,
)

trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets["train_advice_story"],
    eval_dataset=tokenized_datasets["valid_advice_story"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

# this should save the model to disk. changing wd so that it saves here:
fit_path = '/content/gdrive/MyDrive/w266/w266_reddit_summarization/data/model_outputs/bart_fitted_models/round2'
os.chdir(fit_path)
trainer.train()

Cloning https://huggingface.co/trevorj/BART_reddit_advice_story into local empty directory.
Using cuda_amp half precision backend
The following columns in the training set don't have a corresponding argument in `BartForConditionalGeneration.forward` and have been ignored: subreddit_group, subreddit, content, summary. If subreddit_group, subreddit, content, summary are not expected by `BartForConditionalGeneration.forward`,  you can safely ignore this message.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 15000
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total opti

Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,3.374300,3.278733,21.127500,5.961800,17.377200,18.317000,20.447000
2,3.025000,3.246588,21.844300,6.235100,17.635800,18.625900,21.506000
3,2.762800,3.255172,21.934900,6.341700,17.713300,18.719900,21.092000


Saving model checkpoint to BART_reddit_advice_story/checkpoint-500
Configuration saved in BART_reddit_advice_story/checkpoint-500/config.json
Model weights saved in BART_reddit_advice_story/checkpoint-500/pytorch_model.bin
tokenizer config file saved in BART_reddit_advice_story/checkpoint-500/tokenizer_config.json
Special tokens file saved in BART_reddit_advice_story/checkpoint-500/special_tokens_map.json
tokenizer config file saved in BART_reddit_advice_story/tokenizer_config.json
Special tokens file saved in BART_reddit_advice_story/special_tokens_map.json
Saving model checkpoint to BART_reddit_advice_story/checkpoint-1000
Configuration saved in BART_reddit_advice_story/checkpoint-1000/config.json
Model weights saved in BART_reddit_advice_story/checkpoint-1000/pytorch_model.bin
tokenizer config file saved in BART_reddit_advice_story/checkpoint-1000/tokenizer_config.json
Special tokens file saved in BART_reddit_advice_story/checkpoint-1000/special_tokens_map.json
Deleting older checkp

CPU times: user 1h 15min 36s, sys: 5min 50s, total: 1h 21min 27s
Wall time: 1h 23min 48s


In [ ]:
%%time
# also save on huggingface
trainer.push_to_hub()
# fit_path = '/content/gdrive/MyDrive/w266/w266_reddit_summarization/data/model_outputs/bart_fitted_models/'
# os.chdir(fit_path)
# trainer.push_to_hub('trevorj/BART-reddit-advice_story')


# then load model back in
# model = AutoModelForSeq2SeqLM.from_pretrained("trevorj/model_name")

# push to hub seems to fail with this message:
# Dropping the following result as it does not have all the necessary fields:
# {'task': {'name': 'Sequence-to-sequence Language Modeling', 'type': 'text2text-generation'}, 'metrics': [{'name': 'Rouge1', 'type': 'rouge', 'value': 20.9206}]}

Saving model checkpoint to BART_reddit_advice_story
Configuration saved in BART_reddit_advice_story/config.json
Model weights saved in BART_reddit_advice_story/pytorch_model.bin
tokenizer config file saved in BART_reddit_advice_story/tokenizer_config.json
Special tokens file saved in BART_reddit_advice_story/special_tokens_map.json
Dropping the following result as it does not have all the necessary fields:
{'task': {'name': 'Sequence-to-sequence Language Modeling', 'type': 'text2text-generation'}, 'metrics': [{'name': 'Rouge1', 'type': 'rouge', 'value': 21.9349}]}


CPU times: user 1.05 s, sys: 1.41 s, total: 2.47 s
Wall time: 4.96 s


Compare Model predictions

In [ ]:
# true:
print("Content:")
pprint(tokenized_datasets['test_advice_story']['content'][0])
print("\n\nTrue Summary")
pprint(tokenized_datasets['test_advice_story']['summary'][0])

Content:
('Living in the Sierra Nevadas, it gets very cold at night. \n'
 ' Took a shower after a cold night at work. Shower with nice hot water. Hot '
 'but not steaming hot. Felt so good and warm. \n'
 ' Drying myself, starting to get cold. \n'
 ' Got out of the shower, so damn cold. \n'
 ' Drying hair(on head) with the blow dryer and it felt so nice and warm.\n'
 'Balls were cold, penis was cold. All shrivled up, smaller than usual. \n'
 ' Blow dry my private area just as usual during a cold winter day, but since '
 'it was so cold I moved in a little closer. It felt so nice and warm. \n'
 'Thought "oh, fuck it, I\'ll move in a little closer since it was so damn '
 'cold today." Moved in too close, tip of penis touches the burning hot iron '
 'grill. Pull dryer away in shock. \n'
 " It hurts. It burns as I type this. Erections make it burn more. It's red "
 "where it got burned. Hoping it doesn't turn into a blister")


True Summary
('Too damn cold, showered, dry my hair, dry my pri

In [ ]:
# results from my trained model above
# note, I had to add .cuda() to the end of the input tensor to specify to use gpu i guess. 
# But dont do this for the original model. just your fine tuned one. 
output = model.generate(torch.tensor([tokenized_datasets['test_advice_story']['input_ids'][0]]).cuda(), num_beams=2, max_length=60, min_length=2, no_repeat_ngram_size=3)
output_decoded = tokenizer.decode(output.squeeze(), skip_special_tokens=True)
pprint(output_decoded)

'Got cold, penis burns.'


In [ ]:
# read model from disk (same prediction. Good) 
# note, using the latest checkpoint produced the same thing, so the weights are likely almost the same. 
# final files in the base folder will get overwritten at very end with final weights. 
# so we can just delete the older checkpoint folders

checkpoint_disk = "/content/gdrive/MyDrive/w266/w266_reddit_summarization/data/model_outputs/bart_fitted_models/round2/BART_reddit_advice_story"
# model_disk = AutoModelForSeq2SeqLM.from_pretrained(checkpoint_disk)
model_disk = AutoModelForSeq2SeqLM.from_pretrained(checkpoint_disk, local_files_only=True) # optional to force looking locally

clear_output()
output = model_disk.generate(torch.tensor([tokenized_datasets['test_advice_story']['input_ids'][0]]), num_beams=2, max_length=60, min_length=2, no_repeat_ngram_size=3)
output_decoded = tokenizer.decode(output.squeeze(), skip_special_tokens=True)
pprint(output_decoded)

'Got cold, penis burns.'


In [ ]:
# compare to original unfit model. (prediction is different, good). 
model_original = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)
clear_output()
output = model_original.generate(torch.tensor([tokenized_datasets['test_advice_story']['input_ids'][0]]), num_beams=2, max_length=60, min_length=2, no_repeat_ngram_size=3)
output_decoded = tokenizer.decode(output.squeeze(), skip_special_tokens=True)
pprint(output_decoded)

(' In a series of letters from African journalists, filmmaker and columnist '
 'Adelisa Fonseca looks back at her recent cold weather.')


In [ ]:
# Load from huggingface (haven't configured yet)
# new_checkpoint = 'trevorj/BART-reddit-advice_story'
# model_advice_story = AutoModelForSeq2SeqLM.from_pretrained(new_checkpoint)
# tokenizer_advice_story = AutoTokenizer.from_pretrained(new_checkpoint)

# Train - media_lifestyle_sports
- 1:12 hrs

In [ ]:
%%time

args = Seq2SeqTrainingArguments(
    f"BART_reddit_media_lifestyle_sports",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8, # 16
    per_device_eval_batch_size=8, #16
    weight_decay=0.01,
    save_total_limit=1, #3,
    num_train_epochs=3, # 1
    predict_with_generate=True,
    fp16=True,
    push_to_hub=True,
)

trainer_media = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets["train_media_lifestyle_sports"],
    eval_dataset=tokenized_datasets["valid_media_lifestyle_sports"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

# this should save the model to disk. changing wd so that it saves here:
fit_path = '/content/gdrive/MyDrive/w266/w266_reddit_summarization/data/model_outputs/bart_fitted_models/round2'
os.chdir(fit_path)
trainer_media.train()

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
Cloning https://huggingface.co/trevorj/BART_reddit_media_lifestyle_sports into local empty directory.
Using cuda_amp half precision backend
The following columns in the training set don't have a corresponding argument in `BartForConditionalGeneration.forward` and have been ignored: subreddit_group, subreddit, content, summary. If subreddit_group, subreddit, content, summary are not expected by `BartForConditionalGeneration.forward`,  you can safely ignore this message.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, 

Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,3.654200,3.548437,15.970400,4.288100,13.462100,13.985700,17.876000


Saving model checkpoint to BART_reddit_media_lifestyle_sports/checkpoint-500
Configuration saved in BART_reddit_media_lifestyle_sports/checkpoint-500/config.json
Model weights saved in BART_reddit_media_lifestyle_sports/checkpoint-500/pytorch_model.bin
tokenizer config file saved in BART_reddit_media_lifestyle_sports/checkpoint-500/tokenizer_config.json
Special tokens file saved in BART_reddit_media_lifestyle_sports/checkpoint-500/special_tokens_map.json
tokenizer config file saved in BART_reddit_media_lifestyle_sports/tokenizer_config.json
Special tokens file saved in BART_reddit_media_lifestyle_sports/special_tokens_map.json
Saving model checkpoint to BART_reddit_media_lifestyle_sports/checkpoint-1000
Configuration saved in BART_reddit_media_lifestyle_sports/checkpoint-1000/config.json
Model weights saved in BART_reddit_media_lifestyle_sports/checkpoint-1000/pytorch_model.bin
tokenizer config file saved in BART_reddit_media_lifestyle_sports/checkpoint-1000/tokenizer_config.json
Speci

Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,3.654200,3.548437,15.970400,4.288100,13.462100,13.985700,17.876000
2,3.271900,3.540086,15.973300,4.227100,13.440100,14.055000,18.020000
3,3.030100,3.562253,16.246300,4.329600,13.772500,14.350400,17.760000


The following columns in the evaluation set don't have a corresponding argument in `BartForConditionalGeneration.forward` and have been ignored: subreddit_group, subreddit, content, summary. If subreddit_group, subreddit, content, summary are not expected by `BartForConditionalGeneration.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1000
  Batch size = 8
Saving model checkpoint to BART_reddit_media_lifestyle_sports/checkpoint-4000
Configuration saved in BART_reddit_media_lifestyle_sports/checkpoint-4000/config.json
Model weights saved in BART_reddit_media_lifestyle_sports/checkpoint-4000/pytorch_model.bin
tokenizer config file saved in BART_reddit_media_lifestyle_sports/checkpoint-4000/tokenizer_config.json
Special tokens file saved in BART_reddit_media_lifestyle_sports/checkpoint-4000/special_tokens_map.json
Deleting older checkpoint [BART_reddit_media_lifestyle_sports/checkpoint-3500] due to args.save_total_limit
Saving model checkpoin

CPU times: user 1h 5min 6s, sys: 4min 42s, total: 1h 9min 48s
Wall time: 1h 12min 49s


In [ ]:
# create huggingface repo
trainer_media.push_to_hub()

# Train gaming
- 1:13 hrs

In [ ]:
%%time

args = Seq2SeqTrainingArguments(
    f"BART_reddit_gaming",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8, # 16
    per_device_eval_batch_size=8, #16
    weight_decay=0.01,
    save_total_limit=1, #3,
    num_train_epochs=3, # 1
    predict_with_generate=True,
    fp16=True,
    push_to_hub=True,
)

trainer_gaming = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets["train_gaming"],
    eval_dataset=tokenized_datasets["valid_gaming"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

# this should save the model to disk. changing wd so that it saves here:
fit_path = '/content/gdrive/MyDrive/w266/w266_reddit_summarization/data/model_outputs/bart_fitted_models/round2'
os.chdir(fit_path)
trainer_gaming.train()

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
Cloning https://huggingface.co/trevorj/BART_reddit_gaming into local empty directory.
Using cuda_amp half precision backend
The following columns in the training set don't have a corresponding argument in `BartForConditionalGeneration.forward` and have been ignored: subreddit_group, content, summary, subreddit. If subreddit_group, content, summary, subreddit are not expected by `BartForConditionalGeneration.forward`,  you can safely ignore this message.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_depre

Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,3.864000,3.775191,17.375400,4.510000,14.676300,15.220000,16.944000
2,3.475500,3.726480,17.806600,4.418800,14.943200,15.539600,18.104000
3,3.262900,3.737317,18.120200,4.604500,15.127300,15.760100,18.208000


Saving model checkpoint to BART_reddit_gaming/checkpoint-500
Configuration saved in BART_reddit_gaming/checkpoint-500/config.json
Model weights saved in BART_reddit_gaming/checkpoint-500/pytorch_model.bin
tokenizer config file saved in BART_reddit_gaming/checkpoint-500/tokenizer_config.json
Special tokens file saved in BART_reddit_gaming/checkpoint-500/special_tokens_map.json
tokenizer config file saved in BART_reddit_gaming/tokenizer_config.json
Special tokens file saved in BART_reddit_gaming/special_tokens_map.json
Saving model checkpoint to BART_reddit_gaming/checkpoint-1000
Configuration saved in BART_reddit_gaming/checkpoint-1000/config.json
Model weights saved in BART_reddit_gaming/checkpoint-1000/pytorch_model.bin
tokenizer config file saved in BART_reddit_gaming/checkpoint-1000/tokenizer_config.json
Special tokens file saved in BART_reddit_gaming/checkpoint-1000/special_tokens_map.json
Deleting older checkpoint [BART_reddit_gaming/checkpoint-500] due to args.save_total_limit
Sa

CPU times: user 1h 5min 19s, sys: 5min 48s, total: 1h 11min 8s
Wall time: 1h 13min 6s


In [ ]:
# create huggingface repo
trainer_gaming.push_to_hub()

Saving model checkpoint to BART_reddit_gaming
Configuration saved in BART_reddit_gaming/config.json
Model weights saved in BART_reddit_gaming/pytorch_model.bin
tokenizer config file saved in BART_reddit_gaming/tokenizer_config.json
Special tokens file saved in BART_reddit_gaming/special_tokens_map.json
Dropping the following result as it does not have all the necessary fields:
{'task': {'name': 'Sequence-to-sequence Language Modeling', 'type': 'text2text-generation'}, 'metrics': [{'name': 'Rouge1', 'type': 'rouge', 'value': 18.1202}]}


# Train other
- 1:17 hrs

In [ ]:
%%time

args = Seq2SeqTrainingArguments(
    f"BART_reddit_other",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8, # 16
    per_device_eval_batch_size=8, #16
    weight_decay=0.01,
    save_total_limit=1, #3,
    num_train_epochs=3, # 1
    predict_with_generate=True,
    fp16=True,
    push_to_hub=True,
)

trainer_other = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets["train_other"],
    eval_dataset=tokenized_datasets["valid_other"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

# this should save the model to disk. changing wd so that it saves here:
fit_path = '/content/gdrive/MyDrive/w266/w266_reddit_summarization/data/model_outputs/bart_fitted_models/round2'
os.chdir(fit_path)
trainer_other.train()

Cloning https://huggingface.co/trevorj/BART_reddit_other into local empty directory.
Using cuda_amp half precision backend
The following columns in the training set don't have a corresponding argument in `BartForConditionalGeneration.forward` and have been ignored: content, subreddit_group, subreddit, summary. If content, subreddit_group, subreddit, summary are not expected by `BartForConditionalGeneration.forward`,  you can safely ignore this message.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 15000
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimizatio

Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,3.788700,3.604443,18.466800,5.182000,15.359000,16.169000,19.341000
2,3.381600,3.562785,18.099800,4.893700,15.017900,15.761500,17.789000
3,3.134000,3.579218,18.570500,5.010700,15.258100,16.082000,19.402000


Saving model checkpoint to BART_reddit_other/checkpoint-500
Configuration saved in BART_reddit_other/checkpoint-500/config.json
Model weights saved in BART_reddit_other/checkpoint-500/pytorch_model.bin
tokenizer config file saved in BART_reddit_other/checkpoint-500/tokenizer_config.json
Special tokens file saved in BART_reddit_other/checkpoint-500/special_tokens_map.json
tokenizer config file saved in BART_reddit_other/tokenizer_config.json
Special tokens file saved in BART_reddit_other/special_tokens_map.json
Saving model checkpoint to BART_reddit_other/checkpoint-1000
Configuration saved in BART_reddit_other/checkpoint-1000/config.json
Model weights saved in BART_reddit_other/checkpoint-1000/pytorch_model.bin
tokenizer config file saved in BART_reddit_other/checkpoint-1000/tokenizer_config.json
Special tokens file saved in BART_reddit_other/checkpoint-1000/special_tokens_map.json
Deleting older checkpoint [BART_reddit_other/checkpoint-500] due to args.save_total_limit
Saving model ch

CPU times: user 1h 11min 53s, sys: 5min 57s, total: 1h 17min 50s
Wall time: 1h 20min 4s


In [ ]:
# create huggingface repo
trainer_other.push_to_hub()

Saving model checkpoint to BART_reddit_other
Configuration saved in BART_reddit_other/config.json
Model weights saved in BART_reddit_other/pytorch_model.bin
tokenizer config file saved in BART_reddit_other/tokenizer_config.json
Special tokens file saved in BART_reddit_other/special_tokens_map.json
Dropping the following result as it does not have all the necessary fields:
{'task': {'name': 'Sequence-to-sequence Language Modeling', 'type': 'text2text-generation'}, 'metrics': [{'name': 'Rouge1', 'type': 'rouge', 'value': 18.5705}]}
